DRWI Pollution Assessment Stage 2  
Notebook 1: Fecth Data
===

This first notebook fetches and prepares all the input data and modeling necessary for the Stage 2 Assessment.

# Installation and Setup

Carefully follow our **[Installation Instructions](README.md#get-started)**, especially including:
- Creating a virtual environment for this repository (step 3)

## Import Python Dependencies

In [1]:
from pathlib import Path

import numpy     as np
import pandas    as pd
import geopandas as gpd

# packages for data requests
import requests
from requests.auth import HTTPBasicAuth
import json

In [2]:
print("Geopandas: ", gpd.__version__)
# print("spatialpandas: ", spd.__version__)
# print("datashader: ", ds.__version__)
# print("pygeos: ", pygeos.__version__)

Geopandas:  0.10.2


In [3]:
sys.path

['/Users/aaufdenkampe/Documents/Python/pollution-assessment/stage2',
 '/Users/aaufdenkampe/opt/anaconda3/envs/drwi_pa/lib/python39.zip',
 '/Users/aaufdenkampe/opt/anaconda3/envs/drwi_pa/lib/python3.9',
 '/Users/aaufdenkampe/opt/anaconda3/envs/drwi_pa/lib/python3.9/lib-dynload',
 '',
 '/Users/aaufdenkampe/opt/anaconda3/envs/drwi_pa/lib/python3.9/site-packages',
 '/Users/aaufdenkampe/Documents/Python/pollution-assessment/src']

In [4]:
!conda-develop /Users/aaufdenkampe/Documents/Python/pollution-assessment/src

path exists, skipping /Users/aaufdenkampe/Documents/Python/pollution-assessment/src
completed operation for: /Users/aaufdenkampe/Documents/Python/pollution-assessment/src


In [7]:
# Custom functions for Pollution Assessment
import pollution_assessment as pa
import pollution_assessment.plot

In [8]:
# Confirm that the plot sub-module is imported
dir(pa)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'plot']

## Set Paths to Input and Output Files with `pathlib`

Use the [pathlib](https://docs.python.org/3/library/pathlib.html) library (built-in to Python 3) to manage paths indpendentely of OS or environment.

This blog post describes `pathlib`'s benefits relative to using the `os` library or manual approaches.
- https://medium.com/@ageitgey/python-3-quick-tip-the-easy-way-to-deal-with-file-paths-on-windows-mac-and-linux-11a072b58d5f

In [19]:
# Find your current working directory, which should be folder for this notebook.
Path.cwd()

PosixPath('/Users/aaufdenkampe/Documents/Python/pollution-assessment/stage2')

In [21]:
# Set your project directory to your local folder for your clone of this repository
project_path = Path.cwd().parent
project_path

PosixPath('/Users/aaufdenkampe/Documents/Python/pollution-assessment')

In [12]:
# Assign relative paths for data folders. End with a slash character, `/`.
pa1_data_folder = Path('stage1/data/')
pa2_mmw_folder  = Path('stage2/DRB_GWLFE/mmw_results/')

# Stage 2 Naming Conventions
NOTE: we changed these slightly from Stage 1, to better facilitate building various options.

Level 1 names, scale:
* `catch` indicates catchment-level data, for the local catchment only
* `reach` indicates reach-level data, which includes all upstream contributions

Level 2 names, model:
* `_base` indicates model baseline outputs (no conservation)
* `_rest` indicates model with restoration reductions
* `_prot` indicates model with protection projects, avoided loads

**Clusters** are geographic units. There are 8 included in the DRB: Poconos-Kittaninny, Upper Lehigh,  New Jersey Highlands, Middle Schuylkill, Schuylkill Highlands, Upstream Suburban Philadelphia, Brandywine-Christina, Kirkwood-Cohansey Aquifer. These priority locations include parts of pristine headwaters and working forests of the upper watershed, farmlands, suburbs, and industrial and urban centers downstream, and the coastal plain where the river and emerging groundwater empties into either the Delaware Bay or the Atlantic Coast.

**Focus areas** are smaller geographic units within clusters. 

# Read Input Data Files


## Read Stage 1 Files for COMIDs & Geographies
- Background: stage1/WikiSRAT_AnalysisViz.ipynb
- Parquet to GeoDataFrame: https://geopandas.readthedocs.io/en/latest/docs/reference/api/geopandas.read_parquet.html

In [22]:
%%time
# read data from parquet files
catch_base_gdf = gpd.read_parquet(project_path / pa1_data_folder /'base_df_catch.parquet')
reach_base_gdf = gpd.read_parquet(project_path / pa1_data_folder /'base_df_reach.parquet')

CPU times: user 1.11 s, sys: 83.8 ms, total: 1.19 s
Wall time: 1.17 s


In [23]:
catch_base_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19496 entries, 1748535 to 932040370
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   tp_load             19496 non-null  float64 
 1   tn_load             19496 non-null  float64 
 2   tss_load            19496 non-null  float64 
 3   catchment_hectares  19496 non-null  float64 
 4   watershed_hectares  19496 non-null  float64 
 5   tp_loadrate_ws      19496 non-null  float64 
 6   tn_loadrate_ws      19496 non-null  float64 
 7   tss_loadrate_ws     19496 non-null  float64 
 8   maflowv             19496 non-null  float64 
 9   geom_catchment      19496 non-null  geometry
 10  cluster             17358 non-null  category
 11  sub_focusarea       186 non-null    Int64   
 12  nord                18870 non-null  Int64   
 13  nordstop            18844 non-null  Int64   
 14  huc12               19496 non-null  category
 15  streamorder       

In [24]:
reach_base_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19496 entries, 1748535 to 932040370
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   tp_conc             16823 non-null  float64 
 1   tn_conc             16823 non-null  float64 
 2   tss_conc            16823 non-null  float64 
 3   catchment_hectares  19496 non-null  float64 
 4   watershed_hectares  19496 non-null  float64 
 5   maflowv             19496 non-null  float64 
 6   geom                19494 non-null  geometry
 7   cluster             17358 non-null  category
 8   sub_focusarea       186 non-null    Int64   
 9   nord                18870 non-null  Int64   
 10  nordstop            18844 non-null  Int64   
 11  huc12               19496 non-null  category
 12  streamorder         19496 non-null  int64   
 13  headwater           19496 non-null  int64   
 14  phase               4082 non-null   category
 15  fa_name           

### Remove Stage 1 results

In [27]:
pa1_catch_vars = ['tp_load', 'tn_load', 'tss_load',
                  'tp_loadrate_ws', 'tn_loadrate_ws', 'tss_loadrate_ws',]
catch_base_gdf.drop(pa1_catch_vars, axis='columns', inplace=True)

In [30]:
pa1_reach_vars = ['tp_conc', 'tn_conc', 'tss_conc',]
reach_base_gdf.drop(pa1_reach_vars, axis='columns', inplace=True)

## Read Stage 2 MMW Results
- CSV to Pandas: 
  - Guide: https://pandas.pydata.org/docs/user_guide/io.html#csv-text-files 
  - Ref:   https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

### Correct WikiSRAT Names for Load Rate

The WikiSRAT microservice output gives `tploadrate_total`, etc., which Sarah D. saved to the `catchment_load_rates.csv` file.

This is a mistake, and it actually is the **local** NHDplus **catchment annual load (kg/ha)**. 

Mike suspected this error and we confirmed it during Stage 1 by comparing to Model My Watershed subbasin modeling output for specific COMDIDs.

Upstream loads are not returned from WikiSRAT, although they are calculated internally to generate average annual stream reach concentrations.
Upstream loads can therfore be calculated by dividing average anual concentrations (mg/L) by mean annual flow (cfs) to get (kg/y) (after including some unit conversions).

### Units (Not Implemented)

Loads are in kg/y.  
Concentrations are in mg/L.

From Mike:
- `Loadrate_total_ws` was my attempt to get the loadrate totals to kg per hectare
  - Fails if mean annual flow (`maflowv` (CFS)) doesn’t exist (-9999).
  - `loadrate_total_ws  = ((loadate_conc * 28.3168 * 31557600 / 1000000) * maflowv) / watershed_hectares`
    - `31557600 = 365.25 * 24 * 60 * 60`
    - 28.3168 liters in a cubic foot
    - 1000000 mg in kg

From: https://www.fws.gov/r5gomp/gom/nhd-gom/NHDPLUS_UserGuide.pdf
> MAFlowV: Mean Annual Flow (cfs) at bottom of flowline as computed by Vogel Method

**INCOMPLETE**

Options for the future:
- https://stackoverflow.com/questions/14688306/adding-meta-information-metadata-to-pandas-dataframe
- https://pandas.pydata.org/docs/reference/api/pandas.Series.attrs.html
- https://stackoverflow.com/questions/39419178/how-can-i-manage-units-in-pandas-data
- https://github.com/hgrecco/pint
- https://towardsdatascience.com/add-metadata-to-your-dataset-using-apache-parquet-75360d2073bd



In [9]:

wikisrat_catchment_loads = pd.read_csv(project_path / pa2_mmw_folder /
                                            'catchment_loading_rates.csv',
                                            index_col = 'comid',
                                            dtype = {
                                                'Source':'category',
                                                'gwlfe_endpoint':'category',
                                                'huc_run_name':'category',
                                                'huc_run_states':'category',
                                                'land_use_source':'category',
                                                'closest_weather_stations':'category',
                                                'stream_layer':'category',
                                                'weather_source':'category',
                                            }
                                           )


In [10]:
wikisrat_catchment_load_rates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21840 entries, 2612780 to 9891532
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   Unnamed: 0                21840 non-null  int64   
 1   TotalN                    21840 non-null  float64 
 2   TotalP                    21840 non-null  float64 
 3   Sediment                  21840 non-null  float64 
 4   gwlfe_endpoint            21840 non-null  category
 5   huc                       21840 non-null  int64   
 6   huc_level                 21840 non-null  int64   
 7   huc_run                   21840 non-null  int64   
 8   huc_run_level             21840 non-null  int64   
 9   huc_run_name              21840 non-null  category
 10  huc_run_states            21840 non-null  category
 11  huc_run_areaacres         21840 non-null  float64 
 12  land_use_source           21840 non-null  category
 13  closest_weather_stations  21840 non-nu

In [11]:
wikisrat_catchment_concs = pd.read_csv(project_path / pa2_mmw_folder /
                                       'reach_concentrations.csv',
                                       index_col = 'comid',
                                       dtype = {
                                           'Source':'category',
                                           'gwlfe_endpoint':'category',
                                           'huc_run_name':'category',
                                           'huc_run_states':'category',
                                           'land_use_source':'category',
                                           'closest_weather_stations':'category',
                                           'stream_layer':'category',
                                           'weather_source':'category',
                                       }
                                      )

In [12]:
wikisrat_catchment_concs.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21840 entries, 2612780 to 9891532
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   Unnamed: 0                21840 non-null  int64   
 1   TotalN                    15834 non-null  float64 
 2   TotalP                    15834 non-null  float64 
 3   Sediment                  15834 non-null  float64 
 4   gwlfe_endpoint            21840 non-null  category
 5   huc                       21840 non-null  int64   
 6   huc_level                 21840 non-null  int64   
 7   huc_run                   21840 non-null  int64   
 8   huc_run_level             21840 non-null  int64   
 9   huc_run_name              21840 non-null  category
 10  huc_run_states            21840 non-null  category
 11  huc_run_areaacres         21840 non-null  float64 
 12  land_use_source           21840 non-null  category
 13  closest_weather_stations  21840 non-nu

In [13]:
# Explore categoricals
wikisrat_catchment_concs.weather_source.unique()

['NASA_NLDAS_2000_2019', 'USEPA_1960_1990']
Categories (2, object): ['NASA_NLDAS_2000_2019', 'USEPA_1960_1990']

## Add Stage 2 data to Stage 1 dataframe

In [14]:
base_catch_gdf['tp_load2'] = wikisrat_catchment_load_rates.TotalP
base_catch_gdf['tn_load2'] = wikisrat_catchment_load_rates.TotalN
base_catch_gdf['tss_load2'] = wikisrat_catchment_load_rates.Sediment

In [15]:
base_catch_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19496 entries, 1748535 to 932040370
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   tp_load             19496 non-null  float64 
 1   tn_load             19496 non-null  float64 
 2   tss_load            19496 non-null  float64 
 3   catchment_hectares  19496 non-null  float64 
 4   watershed_hectares  19496 non-null  float64 
 5   tp_loadrate_ws      19496 non-null  float64 
 6   tn_loadrate_ws      19496 non-null  float64 
 7   tss_loadrate_ws     19496 non-null  float64 
 8   maflowv             19496 non-null  float64 
 9   geom_catchment      19496 non-null  geometry
 10  cluster             17358 non-null  category
 11  sub_focusarea       186 non-null    Int64   
 12  nord                18870 non-null  Int64   
 13  nordstop            18844 non-null  Int64   
 14  huc12               19496 non-null  category
 15  streamorder       

In [16]:
base_reach_gdf['tp_conc2'] = wikisrat_catchment_concs.TotalP
base_reach_gdf['tn_conc2'] = wikisrat_catchment_concs.TotalN
base_reach_gdf['tss_conc2'] = wikisrat_catchment_concs.Sediment

In [19]:
base_reach_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19496 entries, 1748535 to 932040370
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   tp_conc             16823 non-null  float64 
 1   tn_conc             16823 non-null  float64 
 2   tss_conc            16823 non-null  float64 
 3   catchment_hectares  19496 non-null  float64 
 4   watershed_hectares  19496 non-null  float64 
 5   maflowv             19496 non-null  float64 
 6   geom                19494 non-null  geometry
 7   cluster             17358 non-null  category
 8   sub_focusarea       186 non-null    Int64   
 9   nord                18870 non-null  Int64   
 10  nordstop            18844 non-null  Int64   
 11  huc12               19496 non-null  category
 12  streamorder         19496 non-null  int64   
 13  headwater           19496 non-null  int64   
 14  phase               4082 non-null   category
 15  fa_name           

# Other stuff....

In [ ]:

base_sourc_df = pd.read_csv(mmw_data_folder /'wikisrat_catchment_sources.csv')


In [5]:
%%time
# read data from parquet files
base_catch_gdf = gpd.read_parquet(data_folder /'base_df_catch.parquet')
base_reach_gdf = gpd.read_parquet(data_folder /'base_df_reach.parquet')

rest_catch_gdf = gpd.read_parquet(data_folder /'rest_df_catch.parquet')
rest_reach_gdf = gpd.read_parquet(data_folder /'rest_df_reach.parquet')

point_src_gdf = gpd.read_parquet(data_folder /'point_source_df.parquet')

proj_prot_gdf = gpd.read_parquet(data_folder /'prot_proj_df.parquet')
proj_rest_gdf = gpd.read_parquet(data_folder /'rest_proj_df.parquet')

cluster_gdf = gpd.read_parquet(data_folder /'cluster_df.parquet')   

mmw_huc12_loads_df = pd.read_parquet(data_folder /'mmw_huc12_loads_df.parquet')

Wall time: 2.2 s


In [6]:
focusarea_gdf = gpd.read_parquet(data_folder /'fa_phase2_df.parquet')
focusarea_gdf.cluster = focusarea_gdf.cluster.replace('Kirkwood Cohansey Aquifer', 'Kirkwood - Cohansey Aquifer') # update name for consistency with other files 
focusarea_gdf.set_index('name', inplace=True)

Follow this notebook with WikiSRAT_Analysis.ipynb for analysis of fetched data. 